In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Murtaza Safdari <musafdar@cern.ch>, Jongho Lee <jongho.lee@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [ ]:
from beamtest_analysis_helper import etroc2_analysis_helper
import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from natsort import natsorted
from scipy.optimize import curve_fit

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_W36_IP7_13_HV210V_offset20","ET2_EPIR_1_1_HV210V_offset20", "ET2_CNM_1_3_HV210V_offset20"]
chip_fignames = chip_names
chip_figtitles = ["ETROC2 WB W36 IP7-13 HV210V OS:20","ETROC2 BB EPIR 1-1 HV210V OS:20", "(Trigger) ETROC2 BB CNM 1-3 HV210V OS:20"]

chip_labels= ["1","0","3"]

today = datetime.date.today().isoformat()
fig_outdir = Path('../../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# path_pattern = f"*2023-09-21_Array_Test_Results/SelfTrigger_bottom_Readout_topbottom_1"
path_pattern = f"./SelfTrigger_ET2_CNM_BATCH_1_3_Readout_ET2_EPIR_BATCH1_1_ET2_W36_IP7_13_ET2_CNM_BATCH1_3_loop_*.pqt"

helper = etroc2_analysis_helper(chip_names=chip_names, chip_figtitles=chip_figtitles, chip_labels=chip_labels)

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

fig = plt.figure(dpi=50, figsize=(5,5))
gs = fig.add_gridspec(1,1)

ax0 = fig.add_subplot(gs[0,0])
ax0.plot([1, 0], [1, 0])
plt.show()

In [ ]:
files = glob(path_pattern)
files = natsorted(files)

dataframes = []

cal_cut = [150, 250] # min, max
toa_cut = [100, 500] # min, max
tot_cut = [ 50, 300] # min, max

for ifile in files:
    tmp_df = pd.read_parquet(ifile)

    ## Boundary values are included by default:
    selected_df = tmp_df[tmp_df['cal'].between(cal_cut[0], cal_cut[1]) &
                         tmp_df['toa'].between(toa_cut[0], toa_cut[1]) &
                         tmp_df['tot'].between(tot_cut[0], tot_cut[1])]

    selected_df.reset_index(inplace=True, drop=True)

    dataframes.append(selected_df)
    del tmp_df, selected_df

df = pd.concat(dataframes)

In [ ]:
df.info()

In [ ]:
# Group the DataFrame by 'evt' and count unique 'board' values in each group
unique_board_counts = df.groupby('evt')['board'].nunique()

## event has two unique board ID
event_numbers_with_three_unique_boards = unique_board_counts[unique_board_counts == 3].index
subset_df = df[df['evt'].isin(event_numbers_with_three_unique_boards)]
subset_df.reset_index(inplace=True, drop=True)

In [ ]:
## event has one hit from each board
event_board_counts = subset_df.groupby(['evt', 'board']).size().unstack(fill_value=0)
selected_event_numbers = event_board_counts[(event_board_counts[0] == 1) & (event_board_counts[1] == 1) & (event_board_counts[3] == 1)].index
selected_subset_df = subset_df[subset_df['evt'].isin(selected_event_numbers)]
selected_subset_df.reset_index(inplace=True, drop=True)
selected_subset_df
del df

In [ ]:
selected_subset_df.info()

In [ ]:
h_selection = helper.return_hist(selected_subset_df, chip_names, chip_labels)

In [ ]:
helper.make_pix_inclusive_plots(h_selection, chip_names[0], chip_fignames[0], chip_figtitles[0], fig_path, save=False, show=True, tag="inclusive", title_tag=", inclusive")

In [ ]:
def simple_filter(group, row, col):
    return any((group['board'] == 1) & (group['row'] == row) & (group['col'] == col))

def custom_filter(group):
    return any((group['board'] == 1) & (group['row'] == 15) & (group['col'] == 6)) & \
            any((group['board'] == 0) & (group['row'] == 2) & (group['col'] == 6)) & \
            any((group['board'] == 3) & (group['row'] == 2) & (group['col'] == 5))

def sort_filter(group):
    return group.sort_values(by=['board'], ascending=True)

def distance_filter(group):
    board0_row = group[(group["board"] == 0)]
    board3_row = group[(group["board"] == 3)]
    board0_col = group[(group["board"] == 0)]
    board3_col = group[(group["board"] == 3)]

    if not board0_row.empty and not board3_row.empty and not board0_col.empty and not board3_col.empty:
        row_index_diff = abs(board0_row["row"].values[0] - board3_row["row"].values[0])
        col_index_diff = abs(board0_col["col"].values[0] - board3_col["col"].values[0])
        return row_index_diff < 2 and col_index_diff < 2
    else:
        return False

In [ ]:
tmp_group = selected_subset_df.groupby('evt')
filtered_simple_group = tmp_group.filter(simple_filter, row=15, col=6)
filtered_simple_group.reset_index(inplace=True, drop=True)
del tmp_group

In [ ]:
grouped = filtered_simple_group.groupby('evt')
sorted_filtered_simple_group = grouped.apply(sort_filter)
sorted_filtered_simple_group.reset_index(inplace=True, drop=True)
sorted_filtered_simple_group
del grouped

In [ ]:
grouped = sorted_filtered_simple_group.groupby('evt')
dis_simple_group = grouped.filter(distance_filter)
dis_simple_group

In [ ]:
test_group = dis_simple_group.groupby(['board', 'row', 'col'])
test = test_group.size().reset_index(name='count')
test.to_csv('test.csv', index=False)

In [ ]:
tmp_group = selected_subset_df.groupby('evt')
filtered_group = tmp_group.filter(custom_filter)
filtered_group.reset_index(inplace=True, drop=True)
filtered_group

### Convert txt files to Pandas dataframe

In [ ]:
selected_df = filtered_group

pix_rows = []
pix_cols = []
fit_params = []
cal_means = {boardID:{} for boardID in chip_labels}

for boardID in chip_labels:
    groups = selected_df[selected_df['board'] == int(boardID)].groupby(['row', 'col'])
    for (row, col), group in groups:
        
        cal_mean = group['cal'].mean()
        cal_means[boardID][(row, col)] = cal_mean

In [ ]:
cal_means

In [ ]:
event_based_group = selected_df.groupby('evt')

d = []

for key, data in event_based_group:

    sorted_group = sort_filter(data)
    toa_in_time_b0 = 12.5 - (sorted_group[sorted_group['board'] == 0]['toa'] * (3.125/cal_means["0"][(2, 6)]))
    toa_in_time_b1 = 12.5 - (sorted_group[sorted_group['board'] == 1]['toa'] * (3.125/cal_means["1"][(15, 6)]))
    toa_in_time_b3 = 12.5 - (sorted_group[sorted_group['board'] == 3]['toa'] * (3.125/cal_means["3"][(2, 5)]))

    tot_in_time_b0 = (2*sorted_group[sorted_group['board'] == 0]['tot'] - np.floor(sorted_group[sorted_group['board'] == 0]['tot']/32)) * (3.125/cal_means["0"][(2, 6)])
    tot_in_time_b1 = (2*sorted_group[sorted_group['board'] == 1]['tot'] - np.floor(sorted_group[sorted_group['board'] == 1]['tot']/32)) * (3.125/cal_means["1"][(15, 6)])
    tot_in_time_b3 = (2*sorted_group[sorted_group['board'] == 3]['tot'] - np.floor(sorted_group[sorted_group['board'] == 3]['tot']/32)) * (3.125/cal_means["3"][(2, 5)])

    d.append(
        {
        'evt': key,
        'toa_b0': toa_in_time_b0.values.squeeze(),
        'tot_b0': tot_in_time_b0.values.squeeze(),
        'toa_b1': toa_in_time_b1.values.squeeze(),
        'tot_b1': tot_in_time_b1.values.squeeze(),
        'toa_b3': toa_in_time_b3.values.squeeze(),
        'tot_b3': tot_in_time_b3.values.squeeze(),
        }
    )

df_in_time = pd.DataFrame(d)
del d, selected_df

In [ ]:
df_in_time = df_in_time.astype(float)

## Resolution

In [ ]:
del_toa_b0 = (0.5*(df_in_time['toa_b1'] + df_in_time['toa_b3']) - df_in_time['toa_b0']).values
del_toa_b1 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b3']) - df_in_time['toa_b1']).values
del_toa_b3 = (0.5*(df_in_time['toa_b0'] + df_in_time['toa_b1']) - df_in_time['toa_b3']).values

In [ ]:
coeff_b0 = np.polyfit(df_in_time['tot_b0'].values, del_toa_b0, 3)
poly_func_b0 = np.poly1d(coeff_b0)

coeff_b1 = np.polyfit(df_in_time['tot_b1'].values, del_toa_b1, 3)
poly_func_b1 = np.poly1d(coeff_b1)

coeff_b3 = np.polyfit(df_in_time['tot_b3'].values, del_toa_b3, 3)
poly_func_b3 = np.poly1d(coeff_b3)

In [ ]:
import mplhep as hep
plt.style.use(hep.style.CMS)

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, poly_func_b0(df_in_time['tot_b0'].values), 'r.', label='fit')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, poly_func_b1(df_in_time['tot_b1'].values), 'r.', label='fit')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b3'].values,  del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b3'].values, poly_func_b3(df_in_time['tot_b3'].values), 'r.', label='fit')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
diff_b0 = del_toa_b0 - poly_func_b0(df_in_time['tot_b0'].values)
diff_b1 = del_toa_b1 - poly_func_b1(df_in_time['tot_b1'].values)
diff_b3 = del_toa_b3 - poly_func_b3(df_in_time['tot_b3'].values)

# Define the Gaussian function
def Gauss(x, a, mu, sig):
    y = a*np.exp(-1*(1/(2*sig**2))*(x-mu)**2)
    return y

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b0, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)

axes[0].hist(diff_b0, range=(-1,1), bins=50, density=True, label='')
axes[0].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[0].set_xlabel('Time Walk Corrected TOA [ns]')
axes[0].set_ylabel('Arbitrary Units')
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b1, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)

axes[1].hist(diff_b1, range=(-1,1), bins=50, density=True, label='')
axes[1].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[1].set_xlabel('Time Walk Corrected TOA [ns]')
axes[1].set_ylabel('Arbitrary Units')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b3, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)

axes[2].hist(diff_b1, range=(-1,1), bins=50, density=True, label='')
axes[2].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[2].set_xlabel('Time Walk Corrected TOA [ns]')
axes[2].set_ylabel('Arbitrary Units')
axes[2].legend()

plt.tight_layout()

In [ ]:
diff_b01 = diff_b0 - diff_b1
diff_b03 = diff_b0 - diff_b3
diff_b13 = diff_b1 - diff_b3

fit_params = []

# Define the Gaussian function
def Gauss(x, a, mu, sig):
    y = a*np.exp(-1*(1/(2*sig**2))*(x-mu)**2)
    return y

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 - Board 1', loc="right", size=25)

bins, edges = np.histogram(diff_b01, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[0].hist(diff_b01, range=(-1,1), bins=50, density=True, label='')
axes[0].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[0].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[0].set_ylabel('Arbitrary Units')
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 0 - Board 3', loc="right", size=25)

bins, edges = np.histogram(diff_b03, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[1].hist(diff_b03, range=(-1,1), bins=50, density=True, label='')
axes[1].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[1].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[1].set_ylabel('Arbitrary Units')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 1 - Board 3', loc="right", size=25)

bins, edges = np.histogram(diff_b13, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[2].hist(diff_b13, range=(-1,1), bins=50, density=True, label='')
axes[2].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[2].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[2].set_ylabel('Arbitrary Units')
axes[2].legend()

plt.tight_layout()

In [ ]:
time_resol_b0 = 0.5*(np.sqrt(fit_params[0][2]**2 + fit_params[1][2]**2 - fit_params[2][2]**2))
time_resol_b0*1e3

### Try Neural Network

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input

In [ ]:
def return_dense_model(numpars=2):
    input  = Input(shape=(numpars,), name='input')
    dense1 = Dense(5, activation='relu', name='dense1')(input)
    output = Dense(1, activation='linear', name='output')(dense1)
    model  = Model(inputs=[input], outputs=output, name="simple_dense_NN")
    model.compile(loss='mse', optimizer='adam')
    print(model.summary())
    return model

fit_params_NN = []

In [ ]:
model_b01 = return_dense_model(numpars=2)
checkpointer = ModelCheckpoint('models/weights_b01.hdf5', verbose=1, save_best_only=True,monitor="val_loss")
term = tf.keras.callbacks.TerminateOnNaN()
escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)
history_b01 = model_b01.fit(
    df_in_time[['tot_b0', 'tot_b1']].values, 
    (df_in_time['toa_b0']-df_in_time['toa_b1']).values, 
    validation_split=0.3, 
    epochs=150,
    callbacks=[checkpointer,term,escb])

#plot the loss and validation loss of the dataset
plt.plot(history_b01.history['loss'], label='mse')
plt.plot(history_b01.history['val_loss'], label='val_mse')
plt.yscale("log")
plt.legend()

model_b01.load_weights('models/weights_b01.hdf5')
Y_pred = model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values).flatten()

data_b01 = (df_in_time['toa_b0']-df_in_time['toa_b1']).values-Y_pred

fig, ax = plt.subplots(figsize=(10, 10))
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(f'Board 0 - Board 1', loc="right", size=25)

bins, edges = np.histogram(data_b01, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params_NN.append(popt)

ax.hist(data_b01, range=(-1,1), bins=50, density=True, label='')
ax.plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
ax.set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
ax.set_ylabel('Arbitrary Units')
ax.legend()
print(popt)

In [ ]:
model_b03 = return_dense_model(numpars=2)
checkpointer = ModelCheckpoint('models/weights_b03.hdf5', verbose=1, save_best_only=True,monitor="val_loss")
term = tf.keras.callbacks.TerminateOnNaN()
escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)
history_b03 = model_b03.fit(
    df_in_time[['tot_b0', 'tot_b3']].values, 
    (df_in_time['toa_b0']-df_in_time['toa_b3']).values, 
    validation_split=0.3, 
    epochs=150,
    callbacks=[checkpointer,term,escb])

#plot the loss and validation loss of the dataset
plt.plot(history_b03.history['loss'], label='mse')
plt.plot(history_b03.history['val_loss'], label='val_mse')
plt.yscale("log")
plt.legend()

model_b03.load_weights('models/weights_b03.hdf5')
Y_pred = model_b03.predict(df_in_time[['tot_b0', 'tot_b3']].values).flatten()

data_b03 = (df_in_time['toa_b0']-df_in_time['toa_b3']).values-Y_pred

fig, ax = plt.subplots(figsize=(10, 10))
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(f'Board 0 - Board 3', loc="right", size=25)

bins, edges = np.histogram(data_b03, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params_NN.append(popt)

ax.hist(data_b03, range=(-1,1), bins=50, density=True, label='')
ax.plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
ax.set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
ax.set_ylabel('Arbitrary Units')
ax.legend()
print(popt)

In [ ]:
model_b13 = return_dense_model(numpars=2)
checkpointer = ModelCheckpoint('models/weights_b13.hdf5', verbose=1, save_best_only=True,monitor="val_loss")
term = tf.keras.callbacks.TerminateOnNaN()
escb = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=15, verbose=0)
history_b13 = model_b13.fit(
    df_in_time[['tot_b1', 'tot_b3']].values, 
    (df_in_time['toa_b1']-df_in_time['toa_b3']).values, 
    validation_split=0.3, 
    epochs=150,
    callbacks=[checkpointer,term,escb])

#plot the loss and validation loss of the dataset
plt.plot(history_b13.history['loss'], label='mse')
plt.plot(history_b13.history['val_loss'], label='val_mse')
plt.yscale("log")
plt.legend()

model_b13.load_weights('models/weights_b13.hdf5')
Y_pred = model_b13.predict(df_in_time[['tot_b1', 'tot_b3']].values).flatten()

data_b13 = (df_in_time['toa_b1']-df_in_time['toa_b3']).values-Y_pred

fig, ax = plt.subplots(figsize=(10, 10))
hep.cms.text(loc=0, ax=ax, text="Preliminary", fontsize=25)
ax.set_title(f'Board 1 - Board 3', loc="right", size=25)

bins, edges = np.histogram(data_b13, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params_NN.append(popt)

ax.hist(data_b13, range=(-1,1), bins=50, density=True, label='')
ax.plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
ax.set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
ax.set_ylabel('Arbitrary Units')
ax.legend()
print(popt)

In [ ]:
time_resol_b0 = 0.5*(np.sqrt(fit_params_NN[0][2]**2 + fit_params_NN[1][2]**2 - fit_params_NN[2][2]**2))
time_resol_b0*1e3

In [ ]:
model_b01.load_weights('models/weights_b01.hdf5')
nn_del_b01 = model_b01.predict(df_in_time[['tot_b0', 'tot_b1']].values).flatten()

model_b03.load_weights('models/weights_b03.hdf5')
nn_del_b03 = model_b01.predict(df_in_time[['tot_b0', 'tot_b3']].values).flatten()

model_b13.load_weights('models/weights_b13.hdf5')
nn_del_b13 = model_b01.predict(df_in_time[['tot_b1', 'tot_b3']].values).flatten()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction NN', loc="right", size=25)
axes[0].scatter(df_in_time['tot_b0'].values,  del_toa_b0, label='data')
axes[0].plot(df_in_time['tot_b0'].values, -0.5*(nn_del_b01+nn_del_b03), 'r.', label='Neural Network')
axes[0].set_xlabel('TOT time [ns]')
axes[0].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]' )
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction NN', loc="right", size=25)
axes[1].scatter(df_in_time['tot_b1'].values,  del_toa_b1, label='data')
axes[1].plot(df_in_time['tot_b1'].values, 0.5*(nn_del_b01-nn_del_b13), 'r.', label='Neural Network')
axes[1].set_xlabel('TOT time [ns]')
axes[1].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction NN', loc="right", size=25)
axes[2].scatter(df_in_time['tot_b3'].values,  del_toa_b3, label='data')
axes[2].plot(df_in_time['tot_b3'].values, 0.5*(nn_del_b03+nn_del_b13), 'r.', label='Neural Network')
axes[2].set_xlabel('TOT time [ns]')
axes[2].set_ylabel(r'$(TOA_{i} + TOA_{j})/2 - TOA$ [ns]')
axes[2].legend()

plt.tight_layout()

In [ ]:
diff_b01_nn = del_toa_b0 + 0.5*(nn_del_b01+nn_del_b03)
diff_b03_nn = del_toa_b1 - 0.5*(nn_del_b01-nn_del_b13)
diff_b13_nn = del_toa_b3 - 0.5*(nn_del_b03+nn_del_b13)

fit_params = []

# Define the Gaussian function
def Gauss(x, a, mu, sig):
    y = a*np.exp(-1*(1/(2*sig**2))*(x-mu)**2)
    return y

fig, axes = plt.subplots(1, 3, figsize=(30, 10))
hep.cms.text(loc=0, ax=axes[0], text="Preliminary", fontsize=25)
axes[0].set_title(f'Board 0 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b01_nn, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[0].hist(diff_b01_nn, range=(-1,1), bins=50, density=True, label='')
axes[0].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[0].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[0].set_ylabel('Arbitrary Units')
axes[0].legend()

hep.cms.text(loc=0, ax=axes[1], text="Preliminary", fontsize=25)
axes[1].set_title(f'Board 1 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b03_nn, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[1].hist(diff_b03_nn, range=(-1,1), bins=50, density=True, label='')
axes[1].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[1].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[1].set_ylabel('Arbitrary Units')
axes[1].legend()

hep.cms.text(loc=0, ax=axes[2], text="Preliminary", fontsize=25)
axes[2].set_title(f'Board 3 Time Walk Correction', loc="right", size=25)

bins, edges = np.histogram(diff_b13_nn, range=(-1,1), bins=50, density=True)
centers = 0.5*(edges[1:]+edges[:-1])
popt, _ = curve_fit(Gauss, centers, bins)
fit_params.append(popt)

axes[2].hist(diff_b13_nn, range=(-1,1), bins=50, density=True, label='')
axes[2].plot(np.linspace(-1,1,500), Gauss(np.linspace(-1,1,500), *popt), 'r-', label=fr'$\mu:{popt[1]:.3f}, \sigma: {abs(popt[2]):.3f}$')
axes[2].set_xlabel(r'Time Walk Corrected $\Delta$TOA [ns]')
axes[2].set_ylabel('Arbitrary Units')
axes[2].legend()

plt.tight_layout()